## Librerias y directorios

### Importacion de librerias

In [1]:
# Bibliotecas para Manipulación de Datos y Análisis
import pandas as pd 
import numpy as np 

# Bibliotecas para Manejo de Archivos
import requests 
import zipfile  
import os  

# Visualización de Datos Perdidos
import missingno as msno

# Extracción de Datos Web
from lxml import html

# Fetching de Datos de UC Irvine Machine Learning Repository
from ucimlrepo import fetch_ucirepo

### Creacion de directorios

In [38]:
# creo la estructura de directorios para almacenar los datos y los guardo dentro de un diccionario
paths_directorios = dict(
    # carpeta models
    path_directorio_modelos='../models',
    # carpeta principal "data"
    path_directorio_datos='../data',
    # carpeta "bronze" para los datos en bruto
    path_directorio_datos_bronze='../data/bronze',
    # carpeta "silver" para los datos procesados
    path_directorio_datos_silver='../data/silver',
    # carpeta de datos listos para bi
    path_directorio_datos_gold='../data/gold',
)

In [39]:
# recorro el diccionario con las rutas de los directorios y los creo si no existen
for path in paths_directorios.values():
  print(path)
  try:
    os.mkdir(path)
  except:
    print(f'El directorio {path} ya existe')

../models
El directorio ../models ya existe
../data
El directorio ../data ya existe
../data/bronze
El directorio ../data/bronze ya existe
../data/silver
El directorio ../data/silver ya existe
../data/gold
El directorio ../data/gold ya existe


# Recolección y comprensión de los Datos





## Obtencion de datos

**Opcion 1: Descarga de .csv desde la libreria especializada de uci**

Se evaluara si esta opcion entrega el mismo archivos .csv facilitado para la solucion del desafio.

In [40]:
# Metodo por medio de libreria especializada de ucimlrepo
# fetch dataset
bank_marketing = fetch_ucirepo(id=222)

# data (as pandas dataframes)
X = bank_marketing.data.features
y = bank_marketing.data.targets
full_df_library = pd.concat([X, y], axis=1,ignore_index=True)

# metadata
full_df_library_metadata = bank_marketing.metadata

# variable information
full_df_library_variables = bank_marketing.variables

In [41]:
full_df_library_variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,None,None,no
1,job,Feature,Categorical,Occupation,"type of job (categorical: 'admin.','blue-colla...",None,no
2,marital,Feature,Categorical,Marital Status,"marital status (categorical: 'divorced','marri...",None,no
3,education,Feature,Categorical,Education Level,"(categorical: 'basic.4y','basic.6y','basic.9y'...",None,no
4,default,Feature,Binary,None,has credit in default?,None,no
5,balance,Feature,Integer,None,average yearly balance,euros,no
6,housing,Feature,Binary,None,has housing loan?,None,no
7,loan,Feature,Binary,None,has personal loan?,None,no
8,contact,Feature,Categorical,None,contact communication type (categorical: 'cell...,None,yes
9,day_of_week,Feature,Date,None,last contact day of the week,None,no


In [42]:
full_df_library.columns = full_df_library_variables['name']
full_df_library.rename(columns={'y':'subscribed'}, inplace=True)

In [43]:
full_df_library.shape

(45211, 17)

**Opcion 2: Descarga de archvio .zip**

En caso de que la opcion 1, no entregue el .csv completo, se trabajara desde este .csv

In [44]:
# URL de la página que contiene el enlace de descarga
url = "https://archive.ics.uci.edu/dataset/222/bank+marketing"

# Descargo y parseo el contenido de la página, en caso de error lo capturo e imprimo por pantalla
try:
    response = requests.get(url)
    response.raise_for_status()
    tree = html.fromstring(response.content)
    print("Contenido de la página obtenido correctamente.")
except requests.exceptions.RequestException as e:
    print(f"Error al obtener el contenido de la página: {e}")
    exit()

# Por medio de expresiones xpath obtengo la ubicacion exacta del zip a descargar
zip_url = tree.xpath('//a[contains(@href, ".zip")]/@href')
# si no se encuentra en el arbol html parseado, lo imprimo por pantalla
if not zip_url:
    print("No se encontró ningún enlace .zip en la página.")
    exit()
# concateno el url del zip para poder descargarlo
zip_url = requests.compat.urljoin(url, zip_url[0])
print(f"Enlace de descarga encontrado: {zip_url}")

# Defino el nombre con el que se guardara el archivo
zip_file_principal = "../data/bronze/bank_marketing.zip"

try:
    # Realizo una solicitud GET para descargar el archivo ZIP desde la URL
    zip_response = requests.get(zip_url)
    zip_response.raise_for_status()
    # Guardo el zip descargado y almacenado en la insancia de zip_response
    with open(zip_file_principal, "wb") as file:
        # Escribo el contenido del archivo ZIP descargado en el archivo local
        file.write(zip_response.content)
    print(f"Archivo '{zip_file_principal}' descargado con éxito.")
except requests.exceptions.RequestException as e:
    # Capturo cualquier excepción y muestro un mensaje de error
    print(f"Error al descargar el archivo: {e}")
    # Finalizo la ejecución del programa si ocurrió un error
    exit()

Contenido de la página obtenido correctamente.
Enlace de descarga encontrado: https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Archivo '../data/bronze/bank_marketing.zip' descargado con éxito.


In [45]:
# Descomprimo el archivo .zip en la carpeta "bronze" creada al comienzo
output_dir_zip_principal = "../data/bronze/bank-additional/"
# intento abrir y descomprimir el archivo .zip dentro de la carpeta "bronze", de lo contrario muestro el error
try:
    with zipfile.ZipFile(zip_file_principal, 'r') as zip_ref:
        zip_ref.extractall(output_dir_zip_principal)
    print(f"Archivo '{zip_file_principal}' descomprimido en '{output_dir_zip_principal}'.")
except zipfile.BadZipFile:
    print(f"El archivo '{zip_file_principal}' está corrupto o no es válido.")

Archivo '../data/bronze/bank_marketing.zip' descomprimido en '../data/bronze/bank-additional/'.


In [46]:
# Defino la ruta del archivo ZIP secundario que se encuentra dentro del directorio de salida del ZIP principal
zip_file_secundario =  output_dir_zip_principal + "bank-additional.zip"

# Defino el directorio de salida donde se extraerá el contenido del archivo ZIP secundario
output_dir_zip_secundario = output_dir_zip_principal + "bank-additional"

# Verifico si el archivo ZIP secundario existe en el sistema de archivos
# Si existe lo extraigo en el directorio espesificado dentro de la cartpeta padre
# En casos contrarios muestro los errores por pantalla
if os.path.exists(zip_file_secundario):
    try:
        with zipfile.ZipFile(zip_file_secundario, 'r') as zip_ref:
            zip_ref.extractall(output_dir_zip_secundario)
        print(f"Archivo '{zip_file_secundario}' descomprimido en '{output_dir_zip_secundario}'.")
    except zipfile.BadZipFile:
        print(f"El archivo '{zip_file_secundario}' está corrupto o no es válido.")
else:
    print(f"No se encontró el archivo secundario '{zip_file_secundario}' para descomprimir.")

Archivo '../data/bronze/bank-additional/bank-additional.zip' descomprimido en '../data/bronze/bank-additional/bank-additional'.


In [47]:
# Leo el archivo zip desde su ubicacion
path_csv_from_zip = '../data/bronze/bank-additional/bank-additional/bank-additional/bank-additional-full.csv'
df_from_zip = pd.read_csv(path_csv_from_zip, sep=';')

In [48]:
#muestro las columnas que tengo disponibles
df_from_zip.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [49]:
# renombro la variable target que pertenece a los clientes suscritos al deposito en plazo
df_from_zip.rename(columns={'y':'subscribed'}, inplace=True)

In [50]:
# veo las dimesiones del dataset
df_from_zip.shape

(41188, 21)

In [51]:
# analizando que las dimensiones del dataset facilitado para el desafio difieren brevemente de las que se obtienen al utilizar la libreria
# de ucimlrepo, defino como dataframe el dataset descargado via el zip, el cual si corresponde con el faciliado para el desafio
df = df_from_zip.copy()

### Descubriendo los datos

### Descripcion y estadisticas basicas

Luego se analizaran visaulmente en la etapa de EDA

In [52]:
df.describe(include='all')

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,subscribed
count,41188.00000,41188,41188,41188,41188,41188,41188,41188,41188,41188,...,41188.000000,41188.000000,41188.000000,41188,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188
unique,NaN,12,4,8,3,3,3,2,10,5,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,...,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,10422,24928,12168,32588,21576,33950,26144,13769,8623,...,NaN,NaN,NaN,35563,NaN,NaN,NaN,NaN,NaN,36548
mean,40.02406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.567593,962.475454,0.172963,NaN,0.081886,93.575664,-40.502600,3.621291,5167.035911,NaN
std,10.42125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.770014,186.910907,0.494901,NaN,1.570960,0.578840,4.628198,1.734447,72.251528,NaN
min,17.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.634000,4963.600000,NaN
25%,32.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.344000,5099.100000,NaN
50%,38.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,999.000000,0.000000,NaN,1.100000,93.749000,-41.800000,4.857000,5191.000000,NaN
75%,47.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000,NaN


In [53]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
subscribed         object
dtype: object

### Extraccion de metadatos

**Datos de los clientes**

1 - age (numérico): edad

2 - job: tipo de empleo (categórico: ‘admin.’, ‘blue-collar’, ‘entrepreneur’, ‘housemaid’, ‘management’, ‘retired’, ‘self-employed’, ‘services’, ‘student’, ‘technician’, ‘unemployed’, ‘unknown’)

3 - marital: estado civil (categórico: ‘divorced’, ‘married’, ‘single’, ‘unknown’; nota: ‘divorced’ significa divorciado o viudo)

4 - education (categórico: ‘basic.4y’, ‘basic.6y’, ‘basic.9y’, ‘high.school’, ‘illiterate’, ‘professional.course’, ‘university.degree’, ‘unknown’): nivel educativo

5 - default: ¿tiene crédito en mora? (categórico: ‘no’, ‘yes’, ‘unknown’)

6 - housing: ¿tiene préstamo de vivienda? (categórico: ‘no’, ‘yes’, ‘unknown’)

7 - loan: ¿tiene préstamo personal? (categórico: ‘no’, ‘yes’, ‘unknown’)

**Relacionado con el último contacto de la campaña actual:**

8 - contact: tipo de comunicación del contacto (categórico: ‘cellular’, ‘telephone’)

9 - month: último mes de contacto del año (categórico: ‘jan’, ‘feb’, ‘mar’, …, ‘nov’, ‘dec’)

10 - day_of_week: último día de contacto de la semana (categórico: ‘mon’, ‘tue’, ‘wed’, ‘thu’, ‘fri’)

11 - duration: duración del último contacto, en segundos (numérico). Nota importante: este atributo afecta significativamente la variable objetivo (por ejemplo, si duration=0 entonces y=‘no’). Sin embargo, la duración no se conoce antes de realizar una llamada. Además, después de finalizar la llamada, y obviamente es conocido. Por lo tanto, esta variable solo debería incluirse con fines de referencia y debería descartarse si se busca un modelo predictivo realista.

**Otros atributos:**

12 - campaign: número de contactos realizados durante esta campaña para este cliente (numérico, incluye el último contacto)

13 - pdays: número de días que han pasado desde que el cliente fue contactado por última vez en una campaña anterior (numérico; 999 significa que el cliente no fue contactado previamente)

14 - previous: número de contactos realizados antes de esta campaña para este cliente (numérico)

15 - poutcome: resultado de la campaña de marketing anterior (categórico: ‘failure’, ‘nonexistent’, ‘success’)

**Atributos de contexto social y económico:**

16 - emp.var.rate: tasa de variación del empleo - indicador trimestral (numérico)

17 - cons.price.idx: índice de precios al consumidor - indicador mensual (numérico)

18 - cons.conf.idx: índice de confianza del consumidor - indicador mensual (numérico)

19 - euribor3m: tasa euribor a 3 meses - indicador diario (numérico)

20 - nr.employed: número de empleados - indicador trimestral (numérico)

**Variable de salida (objetivo deseado):**


21 - y - ¿el cliente ha suscrito un depósito a plazo? (binario: ‘yes’, ‘no’)

### Guardado de datasets

In [55]:
# guardo los dataframes con archivo .csv dentro de la carpeta "bronze"
path_csv_from_library = '../data/bronze/df_from_library.csv'
full_df_library.to_csv(path_csv_from_library, index=False)

path_csv_df_from_zip = '../data/bronze/df_from_zip.csv'
df_from_zip.to_csv(path_csv_df_from_zip, index=False)